In [2]:
import pandas as pd
import numpy as np
from numpy import array as a
from random import random
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize
from sklearn.utils import shuffle
import matplotlib.pyplot as plt 


# input dataset
filename1 = "BIG_DWH_Training.csv"
df1 = pd.read_csv( filename1, header=None )
df1.rename(columns={0: 'Index', 1: 'Height', 2: 'Weight', 3:'Gender'}, inplace=True)
df1 = df1.drop(columns = "Index")

data1=df1.replace({'Gender': -1}, 0)
data1 = shuffle(data1)

#normalize data
normalize = preprocessing.StandardScaler().fit(data1.iloc[:, 1:3].values)
new_df1 = normalize.transform(data1.iloc[:, 1:3].values)
data1['Height'] = new_df1[:, 0]
data1['Weight'] = new_df1[:, 1]

#divide data into training and validation sets
train, validation = train_test_split(data1, test_size=0.05)

featuresTrain = train[['Height','Weight']].values.tolist()
featuresTrain = a(featuresTrain)
genderTrain = train[['Gender']].values.tolist()
genderTrain = a(genderTrain)

featuresVal = validation[['Height','Weight']].values.tolist()
featuresVal = a(featuresVal)
genderVal = validation[['Gender']].values.tolist()
genderVal = a(genderVal)

#test data set
filename2 = "DWH_Test.csv"
df2 = pd.read_csv( filename2, header=None )
df2.rename(columns={0: 'Index', 1: 'Height', 2: 'Weight', 3:'Gender'}, inplace=True)
df2 = df2.drop(columns = "Index")

data2=df2.replace({'Gender': -1}, 0)

#normalize data
normalize = preprocessing.StandardScaler().fit(data2.iloc[:, 1:3].values)
new_df2 = normalize.transform(data2.iloc[:, 1:3].values)
data2['Height'] = new_df2[:, 0]
data2['Weight'] = new_df2[:, 1]


In [14]:
class NeuralNetwork():
    
    #fn for initial weight creation of NN
    def create_network(neuron_in, neuron_out):    
        layer = np.random.randn(neuron_out,neuron_in) * np.sqrt(2/neuron_out)  
        return layer
    
    def sigmoid(x):
        return 1.0 / (1.0 + np.exp(-x))
    
    # Forward propagate: takes weights and features and returns activation of each neuron in every layer of NN
    def forward_propogation(features, ann1, ann2, ann3):
        activation = [] #list
        activation1 = []
        activation2 = []
        
        for row in features:
            act1 = np.dot(ann1, row)
            act1 = NeuralNetwork.sigmoid(act1) 
            activation1.append(act1)
            
            act2 = np.dot(ann2, act1)
            act2 = NeuralNetwork.sigmoid(act2)
            activation2.append(act2)
            
            act3 = np.dot(ann3, act2)
            act3 = NeuralNetwork.sigmoid(act3)
            activation.append(act3)
            
        activation1 = a(activation1)
        activation2 = a(activation2)
        activation = a(activation)

        return activation1, activation2, activation
    
    def sigmoid_derivative(x):
        return x * (1 - x)
    
    # Backpropagate : Computes gradient of each layer and error of every iteration
    def backward_propagation(ann1, ann2, ann3, actual, a1, a2, a3):
        
        error = []
        dError = []
        sigDer3 = []
        delta3 = []
        gradient3=[]
        delta2 = []
        gradient2=[]
        sigDer2=[]
        delta1 = []
        gradient1=[]
        sigDer1=[]
        
        for i in range(len(actual)):
            
            error.append(-(np.multiply(actual[i],np.log(a3[i])) + np.multiply(1-actual[i],np.log(1-a3[i]))))
            dError.append(-((actual[i]/a3[i]) - ((1-actual[i])/(1-a3[i]))))
            sigDer3.append(NeuralNetwork.sigmoid_derivative(a3[i]))
            sigDer2.append(NeuralNetwork.sigmoid_derivative(a2[i]))
            sigDer1.append(NeuralNetwork.sigmoid_derivative(a1[i]))
            delta3.append(dError[i] * sigDer3[i])
           
        error= np.mean(error)

        sigDer2 = a(sigDer2)  
        sigDer1 = a(sigDer1)
        
        delta3 = a(delta3)      
        gradient3 = np.multiply(delta3,a3)
        gradient3 = np.divide(np.mean(gradient3),50)
        
        delta2 = np.dot(delta3,ann3)
        delta2 = np.multiply(delta2,sigDer2)
        gradient2 = np.multiply(delta2,a2)
        gradient2 = np.divide(np.mean(gradient2),50)
   
        delta1 = np.dot(delta2,ann2)
        delta1 = np.multiply(delta1,sigDer1)
        gradient1 = np.multiply(delta1,a1)
        gradient1 = np.divide(np.mean(gradient1),50)
        
        return gradient1, gradient2, gradient3, error   
    
    
    #updates weights based on weights, gradients, and learning rate
    def update_weights(ann1, ann2, ann3, lr, g1, g2, g3): 

        term1 = lr*g1
        term2 = lr*g2
        term3 = lr*g3
        weights1 = np.subtract(ann1, term1)
        weights2 = np.subtract(ann2, term2)
        weights3 = np.subtract(ann3, term3)
        
        return weights1, weights2, weights3

    
    
if __name__ == "__main__":
    
    ann1 = NeuralNetwork.create_network(2,5)
    ann2 = NeuralNetwork.create_network(5,5) 
    ann3 = NeuralNetwork.create_network(5,1)
    
    batchSizeTrain = 0
    batchSizeVal = 0
    valErrorList = np.empty(0)
    trainAccuracy = np.empty(0)
    valAccuracy = np.empty(0)
    out = True
    
    for i in range(1,1000):
        if i%250 != 0:  
            
#             print("Inside Training batch", i)
            
            featuresTrainSplit = featuresTrain[batchSizeTrain:batchSizeTrain+50]
            genderTrainSplit = genderTrain[batchSizeTrain:batchSizeTrain+50]
            activation1, activation2, activation3 = NeuralNetwork.forward_propogation(featuresTrainSplit, ann1, ann2, ann3)
            avg_gradient1, avg_gradient2, avg_gradient3, avg_error = NeuralNetwork.backward_propagation(ann1, ann2, ann3, genderTrainSplit, activation1, activation2, activation3)
#             print("Error :", avg_error)
            
            resultTrain = 0
            predictTrain= np.zeros(50)
            
            for index in range(len(activation3)): 
                if (activation3[index] < 0.5) :      #female
                    predictTrain[index] = 0
                    if (genderTrainSplit[index] == predictTrain[index]): 
                        resultTrain += 1                      
                else:               #classify as male
                    predictTrain[index] = 1    
                    if (genderTrainSplit[index] == predictTrain[index]): 
                        resultTrain += 1
            t_acc = (resultTrain/50) * 100
#             print("Accuracy : ", t_acc ,"%")
#             print("")
            trainAccuracy = np.append(trainAccuracy, t_acc)
            
            ann1, ann2, ann3 = NeuralNetwork.update_weights(ann1, ann2, ann3, 5/i, avg_gradient1, avg_gradient2, avg_gradient3)
            batchSizeTrain += 50
            
        else:   
#             print("")
#             print("Inside Validation batch", i)
            featuresValSplit = featuresVal[batchSizeVal:batchSizeVal+50]
            genderValSplit = genderVal[batchSizeVal:batchSizeVal+50]
            activation1, activation2, activation3 = NeuralNetwork.forward_propogation(featuresValSplit, ann1, ann2, ann3)
            avg_gradient1, avg_gradient2, avg_gradient3, avg_error = NeuralNetwork.backward_propagation(ann1, ann2, ann3, genderValSplit, activation1, activation2, activation3)
#             print("Error: ", avg_error)

            
            if len(valErrorList) % 5 == 0:
                print("Last 5 error: ", valErrorList[-5:])
                print("Average error: ", avg_error)
                out = np.greater(valErrorList[-5:], avg_error)
                if False in out:
                    break
                    
            valErrorList = np.append(valErrorList, avg_error)
                         
            resultVal = 0
            predictVal = np.zeros(50)
            
            for index in range(len(activation3)): 
                if (activation3[index] < 0.5) :      #female
                    predictVal[index] = 0
                    if (genderValSplit[index] == predictVal[index]): 
                        resultVal += 1                      
                else:               #classify as male
                    predictVal[index] = 1    
                    if (genderValSplit[index] == predictVal[index]): 
                        resultVal += 1
            v_acc = (resultVal/50) * 100 
#             print("Accuracy : ", v_acc ,"%")
#             print("")
            
            valAccuracy = np.append(valAccuracy, v_acc)
            
            ann1, ann2, ann3 = NeuralNetwork.update_weights(ann1, ann2, ann3, 5/i, avg_gradient1, avg_gradient2, avg_gradient3)
            batchSizeVal += 50
            
    print("\nTraining accuracies:", trainAccuracy)
    print("Validation accuracies:", valAccuracy)
    print("End of training. Weights found to be as below:- ")
    print("\nLayer 1:\n ", ann1, " \n Layer 2:\n ",ann2,"", "\n Layer 3: \n", ann3)
    
#     #simple graph plot of accuracies
#     # x axis values 
#     x = trainAccuracy 
#     # corresponding y axis values 
#     y = valAccuracy 

#     # plotting the points  
#     plt.plot(x) 

#     # naming the x axis 
#     plt.xlabel('x - axis') 
#     # naming the y axis 
#     plt.ylabel('y - axis') 

#     # giving a title to my graph 
#     plt.title('Training and Validation accuraccies') 

#     # function to show the plot 
#     plt.show() 
    

(50, 1)
(50, 1)
(50, 1)
(50, 1)

Training accuracies: [50. 52. 54. 54.]
Validation accuracies: []
End of training. Weights found to be as below:- 

Layer 1:
  [[ 0.39931276 -0.21889181]
 [-0.1068288  -0.30496878]
 [-0.45343286  0.23089769]
 [ 0.31493661 -0.3268117 ]
 [-0.59686982  0.38485152]]  
 Layer 2:
  [[-0.36289102 -0.04581594  0.95828906  0.20357033 -0.47006621]
 [ 0.52168981  0.33109184 -0.01054542 -0.70416093 -0.08964089]
 [-0.2463336  -0.32992994 -0.25979413 -0.01808637 -0.59831496]
 [ 0.54237695 -0.8896685  -0.99805062 -0.95997024  0.07142816]
 [ 0.71284468  0.528435    0.67593766 -0.6393647   1.05608375]]  
 Layer 3: 
 [[-1.88940393  1.8877302  -0.79013443  0.88625507 -0.39185512]]
